In [1]:
import json
import os
import pickle
import shutil
import sys
import time
from datetime import timedelta, datetime

import igraph as ig
import numpy as np
import pandas as pd
import xgboost as xgb
from pyspark.sql import functions as sf
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
from sklearn.metrics import f1_score, recall_score
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

import settings_medium_hi as s

os.environ["EXT_DATA_TYPE_FOLDER"] = s.OUTPUT_POSTFIX.lstrip("-")

from common import get_weights
from communities import get_communities_spark
from features import (
    generate_features_spark, generate_features_udf_wrapper, get_edge_features_udf, 
    SCHEMA_FEAT_UDF, FEATURE_TYPES
)


%load_ext autoreload
%autoreload 2

In [2]:
shutil.rmtree("artifacts", ignore_errors=True)

config = [
    ("spark.driver.memory", "32g"),
    ("spark.worker.memory", "32g"),
    ("spark.driver.maxResultSize", "32g"),
    ("spark.driver.bindAddress", "127.0.0.1"),
    ("spark.sql.execution.arrow.pyspark.enabled", "true"),
]
spark = (
    SparkSession.builder.appName("testing")
    .config(conf=SparkConf().setAll(config))
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/05 23:29:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
start = time.time()

In [4]:
WINDOW_SIZE = 7
TRAIN_PERC = 0.6
VALIDATION_PERC = 0.2
TEST_PERC = 0.2

NUM_PROCS = 10

assert(sum([TRAIN_PERC, VALIDATION_PERC, TEST_PERC]) == 1)

In [5]:
location_main = os.path.join("features", os.environ["EXT_DATA_TYPE_FOLDER"])
# shutil.rmtree(location_main, ignore_errors=True)

location_flow_dispense = f"{location_main}{os.sep}flow_dispense.parquet"
location_flow_passthrough = f"{location_main}{os.sep}flow_passthrough.parquet"
location_flow_sink = f"{location_main}{os.sep}flow_sink.parquet"

location_comm_as_source_features = f"{location_main}{os.sep}comm_as_source_features.parquet"
location_comm_as_target_features = f"{location_main}{os.sep}comm_as_target_features.parquet"
location_comm_as_passthrough_features = f"{location_main}{os.sep}comm_as_passthrough_features.parquet"
location_comm_as_passthrough_features_reverse = f"{location_main}{os.sep}comm_as_passthrough_features_reverse.parquet"

location_features_node_level = f"{location_main}{os.sep}features_node_level.parquet"
location_features_edges = f"{location_main}{os.sep}features_edges.parquet"

location_features_edges_train = f"{location_main}{os.sep}features_edges_train.parquet"
location_features_edges_valid = f"{location_main}{os.sep}features_edges_valid.parquet"
location_features_edges_test = f"{location_main}{os.sep}features_edges_test.parquet"

location_train_trx_features = f"{location_main}{os.sep}train_trx_features"
location_valid_trx_features = f"{location_main}{os.sep}valid_trx_features"
location_test_trx_features = f"{location_main}{os.sep}test_trx_features"

try:
    os.makedirs(location_main)
except FileExistsError:
    pass

In [6]:
data = spark.read.parquet(s.STAGED_DATA_LOCATION)
data_count_original = data.count()

In [7]:
%%time

KEEP_TOP_N = 100

data_agg_weights = get_weights(
    data.groupby(["source", "target"])
    .agg(
        sf.sum("amount").alias("amount")
    ).toPandas()
)
data_agg_weights.sort_values("weight", ascending=False, inplace=True)

edges_to_keep = data_agg_weights.groupby("source").head(KEEP_TOP_N).reset_index(drop=True)
edges_to_keep.sort_values("weight", ascending=False, inplace=True)
edges_to_keep = edges_to_keep.groupby("target").head(KEEP_TOP_N).reset_index(drop=True)
edges_to_keep = edges_to_keep.loc[:, ["source", "target"]].drop_duplicates()
edges_to_keep = spark.createDataFrame(edges_to_keep)

data_graph = data.join(
    edges_to_keep.select(sf.col("source").alias("src"), sf.col("target").alias("dst")),
    (sf.col("source") == sf.col("src")) &
    (sf.col("target") == sf.col("dst"))
).drop("src", "dst").persist(StorageLevel.DISK_ONLY)
data_count_graph = data_graph.count()
reduction = round((data_count_graph / data_count_original) * 100, 2)
print(f"\nReduced to {reduction}%\n")

[Stage 11:==================================================>  (190 + 10) / 200]


Reduced to 90.77%

CPU times: user 34.6 s, sys: 688 ms, total: 35.3 s
Wall time: 1min 19s


In [8]:
%%time

left = data_graph.select("source", "target", "timestamp", "amount")
select = []
for column in left.columns:
    select.append(sf.col(column).alias(f"left_{column}"))
left = left.select(*select)
right = data_graph.select("source", "target", "timestamp", "amount")

flows_temporal = left.join(
    right,
    (left["left_target"] == right["source"]) &
    (left["left_timestamp"] <= right["timestamp"]),
    how="inner"
).groupby(["left_source", "left_target", "source", "target"]).agg(
    sf.sum("left_amount").alias("left_amount"),
    sf.sum("amount").alias("amount"),
).drop("left_target").select(
    sf.col("left_source").alias("dispense"),
    sf.col("source").alias("passthrough"),
    sf.col("target").alias("sink"),
    sf.least("left_amount", "amount").alias("amount"),
).persist(StorageLevel.DISK_ONLY)
flows_temporal.count()
flows_temporal = flows_temporal.toPandas()

# TODO: This can be made much faster!
flow_dispense, flow_passthrough, flow_sink = [], [], []
for flow_data, flow_type in [
    (flow_dispense, "dispense"), (flow_passthrough, "passthrough"), (flow_sink, "sink")
]:
    print(flow_type)
    prefix = f"{s.G_FLOW_PREFIX}{flow_type}_"
    for key, group in flows_temporal.groupby(flow_type):
        cycle = group[(group["dispense"] == group["sink"]) & (group["dispense"] != group["passthrough"])]
        row = {
            "key": key,
            f"{prefix}amount_sum": group["amount"].sum(),
            f"{prefix}amount_mean": group["amount"].mean(),
            f"{prefix}amount_max": group["amount"].max(),
            f"{prefix}dispense_count": group["dispense"].nunique(),
            f"{prefix}passthrough_count": group["passthrough"].nunique(),
            f"{prefix}sink_count": group["sink"].nunique(),
            f"{prefix}cycle_sum": cycle["amount"].sum(),
            f"{prefix}cycle_mean": cycle["amount"].mean(),
            f"{prefix}cycle_max": cycle["amount"].max(),
            f"{prefix}cycle_passthrough_count": cycle["passthrough"].nunique(),
        }
        flow_data.append(row)

pd.DataFrame(flow_dispense).set_index("key").to_parquet(location_flow_dispense)
pd.DataFrame(flow_passthrough).set_index("key").to_parquet(location_flow_passthrough)
pd.DataFrame(flow_sink).set_index("key").to_parquet(location_flow_sink)

del flows_temporal
del flow_dispense
del flow_passthrough
del flow_sink

dispense
passthrough
sink
CPU times: user 23min 10s, sys: 1min 11s, total: 24min 21s
Wall time: 23min 52s


In [9]:
%%time

trx_ids_sorted = data.sort("timestamp").select("transaction_id").toPandas()["transaction_id"].values
trx_count = len(trx_ids_sorted)
print(trx_count)

last_train_index = int(np.floor(trx_count * TRAIN_PERC))
last_validation_index = last_train_index + int(np.floor(trx_count * VALIDATION_PERC))
train_indexes = trx_ids_sorted[:last_train_index]
validation_indexes = trx_ids_sorted[last_train_index:last_validation_index]
test_indexes = trx_ids_sorted[last_validation_index:]

train_indexes = spark.createDataFrame(
    pd.DataFrame(train_indexes, columns=["transaction_id"])
).persist(StorageLevel.DISK_ONLY)
validation_indexes = spark.createDataFrame(
    pd.DataFrame(validation_indexes, columns=["transaction_id"])
).persist(StorageLevel.DISK_ONLY)
test_indexes = spark.createDataFrame(
    pd.DataFrame(test_indexes, columns=["transaction_id"])
).persist(StorageLevel.DISK_ONLY)

train = train_indexes.join(
    data, on="transaction_id", how="left"
)
validation = validation_indexes.join(
    data, on="transaction_id", how="left"
)
test = test_indexes.join(
    data, on="transaction_id", how="left"
)
train_validation = train.union(validation)

31867069
CPU times: user 331 ms, sys: 137 ms, total: 468 ms
Wall time: 7.75 s


In [10]:
%%time

data_input = data.select("*")
nodes_source = set(data.select("source").distinct().toPandas()["source"])
nodes_target = set(data.select("target").distinct().toPandas()["target"])
nodes_passthrough = nodes_source.intersection(nodes_target)

%run generate_flow_features.ipynb

comm_as_source_features.to_parquet(location_comm_as_source_features)
comm_as_target_features.to_parquet(location_comm_as_target_features)
comm_as_passthrough_features.to_parquet(location_comm_as_passthrough_features)
comm_as_passthrough_features_reverse.to_parquet(location_comm_as_passthrough_features_reverse)

del comm_as_source_features
del comm_as_target_features
del comm_as_passthrough_features
del comm_as_passthrough_features_reverse


Processing comm_as_source

Processed hop #1 | 2,842,770 | 1,333,420
Processed hop #2 | 4,358,267 | 847,080
Processed hop #3 | 6,468,686 | 661,588
Processed hop #4 | 8,492,595 | 593,472

Processing comm_as_target

Processed hop #1 | 2,967,961 | 1,169,331
Processed hop #2 | 11,502,196 | 963,178
Processed hop #3 | 15,539,593 | 919,376
Processed hop #4 | 20,254,210 | 894,217

Processing comm_as_passthrough

Processed hop #1 | 2,405,456 | 949,800
Processed hop #2 | 3,599,750 | 608,779
Processed hop #3 | 5,394,064 | 508,228
Processed hop #4 | 6,981,617 | 456,019

Processing comm_as_passthrough_reverse

Processed hop #1 | 2,811,504 | 1,108,680
Processed hop #2 | 10,885,591 | 912,988
Processed hop #3 | 14,750,939 | 870,897
Processed hop #4 | 19,341,306 | 847,016


comm_as_source_features

CPU times: user 1min 28s, sys: 3.88 s, total: 1min 32s
Wall time: 1min 32s

comm_as_target_features

CPU times: user 1min 42s, sys: 1.96 s, total: 1min 44s
Wall time: 1min 43s

comm_as_passthrough_features



In [11]:
%%time

ts_min = data_graph.select(sf.min("timestamp").alias("x")).collect()[0]["x"] - timedelta(minutes=1)
data_graph_agg = data_graph.groupby(["source", "target", "source_bank", "target_bank", "source_currency"]).agg(
    sf.count("source").alias("num_transactions"),
    sf.sum("amount").alias("amount"),
    sf.sum("source_amount").alias("source_amount"),
    sf.collect_list(sf.array((sf.col("timestamp") - ts_min).cast("long"), sf.col("amount"))).alias("timestamps_amounts"),
)
data_graph_agg_sdf = data_graph_agg.persist(StorageLevel.DISK_ONLY)
_ = data_graph_agg_sdf.count()
data_graph_agg = data_graph_agg_sdf.toPandas().convert_dtypes()

CPU times: user 4.73 s, sys: 1.11 s, total: 5.84 s
Wall time: 14.4 s


In [12]:
%%time

print("Constructing communities")

in_scope_nodes = list(set(data_graph_agg["source"].unique()).union(data_graph_agg["target"].unique()))
window_edges = get_weights(
    data_graph_agg.groupby(["source", "target"]).agg(amount=("amount", "sum")).reset_index()
)
graph = ig.Graph.DataFrame(window_edges, use_vids=False, directed=True)
communities = get_communities_spark(in_scope_nodes, graph, NUM_PROCS, spark)

del in_scope_nodes
del window_edges
del graph

Constructing communities


CPU times: user 45.7 s, sys: 1.52 s, total: 47.2 s
Wall time: 13min 2s


In [13]:
%%time

print("Communities features creation")

graph = ig.Graph.DataFrame(data_graph_agg, use_vids=False, directed=True)
features = generate_features_spark(communities, graph, spark)
features.columns = [f"{s.G_COMM_PREFIX}{x}" if x != "key" else x for x in features.columns]

del graph
del communities
del data_graph_agg

Communities features creation


CPU times: user 24min 7s, sys: 10min 25s, total: 34min 33s
Wall time: 1h 11min 6s


In [14]:
%%time

print("1-hop-source features creation")

features_source = data_graph_agg_sdf.withColumn("key", sf.col("source")).groupby("key").applyInPandas(
    generate_features_udf_wrapper(False), schema=SCHEMA_FEAT_UDF
).toPandas()
features_source = pd.DataFrame(features_source["features"].apply(json.loads).tolist())
types = {k: v for k, v in FEATURE_TYPES.items() if k in features_source.columns}
features_source = features_source.astype(types)
features_source.columns = [f"{s.G_1HOP_PREFIX}{x}" if x != "key" else x for x in features_source.columns]

1-hop-source features creation


CPU times: user 10.6 s, sys: 1.15 s, total: 11.8 s
Wall time: 29min 39s


In [15]:
%%time

print("1-hop-target features creation")

features_target = data_graph_agg_sdf.withColumn("key", sf.col("target")).groupby("key").applyInPandas(
    generate_features_udf_wrapper(False), schema=SCHEMA_FEAT_UDF
).toPandas()
features_target = pd.DataFrame(features_target["features"].apply(json.loads).tolist())
types = {k: v for k, v in FEATURE_TYPES.items() if k in features_target.columns}
features_target = features_target.astype(types)
features_target.columns = [f"{s.G_1HOP_PREFIX}{x}" if x != "key" else x for x in features_target.columns]

1-hop-target features creation


CPU times: user 8.86 s, sys: 1.06 s, total: 9.92 s
Wall time: 25min 5s


In [16]:
%%time

all_features = features.set_index("key").join(
    features_source.set_index("key"), how="left", rsuffix=f"_1_hop_as_source"
)
all_features.index.name = "key"
all_features = all_features.reset_index()

all_features = all_features.set_index("key").join(
    features_target.set_index("key"), how="left", rsuffix=f"_1_hop_as_target"
)

all_features = all_features.join(
    pd.read_parquet(location_comm_as_source_features), how="left", rsuffix="_dispense"
).join(
    pd.read_parquet(location_comm_as_target_features), how="left", rsuffix="_sink"
).join(
    pd.read_parquet(location_comm_as_passthrough_features), how="left", rsuffix="_passthrough"
).join(
    pd.read_parquet(location_comm_as_passthrough_features_reverse), how="left", rsuffix="_passthrough_rev"
).join(
    pd.read_parquet(location_flow_dispense), how="left"
).join(
    pd.read_parquet(location_flow_passthrough), how="left"
).join(
    pd.read_parquet(location_flow_sink), how="left"
)

all_features.to_parquet(location_features_node_level)
del all_features

CPU times: user 20.5 s, sys: 4.53 s, total: 25 s
Wall time: 23.4 s


In [17]:
all_features = pd.read_parquet(location_features_node_level)

In [18]:
anomalies = all_features.loc[:, []]
anomalies.loc[:, "anomaly_score"] = IsolationForest().fit(
    all_features.fillna(0)
).decision_function(all_features.fillna(0))
anomalies.loc[:, "anomaly_score"] += abs(anomalies.loc[:, "anomaly_score"].min())

In [19]:
if s.FILE_SIZE == "Small":
    n_components = 50
elif s.FILE_SIZE == "Medium":
    n_components = 20
else:
    n_components = 5

pca = PCA(n_components=n_components)
all_features_dim_reduced = pd.DataFrame(
    pca.fit_transform(normalize(all_features.fillna(0), norm="l1", axis=1)),
    index=all_features.index
)
print(n_components, round(sum(pca.explained_variance_ratio_) * 100, 2))
all_features_dim_reduced.columns = [
    f"pca_{x + 1}" for x in all_features_dim_reduced.columns
]
del all_features

20 99.87


In [20]:
%%time

print(f"Generating edge features")

to_select = ["source", "target", "format", "source_currency", "source_amount", "amount"]

edges_features_input = data.select(to_select).groupby(
    ["source", "target", "format", "source_currency"]
).agg(
    sf.sum("source_amount").alias("source_amount"), sf.sum("amount").alias("amount")
).persist(StorageLevel.DISK_ONLY)
_ = edges_features_input.count()

edge_features = edges_features_input.groupby(["source", "target"]).applyInPandas(
    get_edge_features_udf, schema=SCHEMA_FEAT_UDF
).toPandas()
edge_features = pd.DataFrame(edge_features["features"].apply(json.loads).tolist())

edge_features.to_parquet(location_features_edges)
del edge_features

Generating edge features


CPU times: user 14.3 s, sys: 3.01 s, total: 17.3 s
Wall time: 16min 32s


In [21]:
edge_features = pd.read_parquet(location_features_edges)

In [22]:
%%time

train_edges = train.select("source", "target").drop_duplicates().toPandas().set_index(
    ["source", "target"]
)
valid_edges = validation.select("source", "target").drop_duplicates().toPandas().set_index(
    ["source", "target"]
)
test_edges = test.select("source", "target").drop_duplicates().toPandas().set_index(
    ["source", "target"]
)

train_features = train_edges.join(
    edge_features.set_index(["source", "target"]), how="left"
).reset_index()
validation_features = valid_edges.join(
    edge_features.set_index(["source", "target"]), how="left"
).reset_index()
test_features = test_edges.join(
    edge_features.set_index(["source", "target"]), how="left"
).reset_index()

25/07/06 02:48:12 WARN TaskSetManager: Stage 119 contains a task of very large size (17125 KiB). The maximum recommended task size is 1000 KiB.
25/07/06 02:48:14 WARN TaskSetManager: Stage 121 contains a task of very large size (17125 KiB). The maximum recommended task size is 1000 KiB.
25/07/06 02:48:21 WARN TaskSetManager: Stage 129 contains a task of very large size (17125 KiB). The maximum recommended task size is 1000 KiB.
25/07/06 02:48:24 WARN TaskSetManager: Stage 131 contains a task of very large size (17125 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

CPU times: user 24.3 s, sys: 1.89 s, total: 26.2 s
Wall time: 2min 48s


In [23]:
def save_edge_features(features_in, location):
    features_in = features_in.set_index("target").join(
        anomalies, how="left"
    ).reset_index().set_index("source").join(
        anomalies, how="left", rsuffix="_source"
    ).reset_index().set_index("target").join(
        all_features_dim_reduced, how="left"
    ).reset_index().set_index("source").join(
        all_features_dim_reduced, how="left", rsuffix="_source"
    ).reset_index()
    features_in.loc[:, "anom_scores_diff"] = features_in.loc[:, "anomaly_score"] - features_in.loc[:, "anomaly_score_source"]
    features_in.loc[:, "anom_scores_min"] = np.array(
        [
            features_in.loc[:, "anomaly_score"].values, 
            features_in.loc[:, "anomaly_score_source"].values
        ],
    ).min(axis=0)
    features_in.loc[:, "anom_scores_max"] = np.array(
        [
            features_in.loc[:, "anomaly_score"].values, 
            features_in.loc[:, "anomaly_score_source"].values
        ],
    ).max(axis=0)
    features_in.loc[:, "anom_scores_mean"] = np.array(
        [
            features_in.loc[:, "anomaly_score"].values, 
            features_in.loc[:, "anomaly_score_source"].values
        ],
    ).mean(axis=0)
    features_in.to_parquet(location)

In [24]:
%%time

save_edge_features(train_features, location_features_edges_train)

CPU times: user 16.4 s, sys: 3.36 s, total: 19.7 s
Wall time: 19.8 s


In [25]:
%%time

save_edge_features(validation_features, location_features_edges_valid)

CPU times: user 5.81 s, sys: 630 ms, total: 6.44 s
Wall time: 6.24 s


In [26]:
%%time

save_edge_features(test_features, location_features_edges_test)

CPU times: user 7.3 s, sys: 942 ms, total: 8.25 s
Wall time: 7.96 s


In [27]:
def save_trx_features(data_in, location):
    columns = ["source", "target", "source_currency", "target_currency", "format", "amount", "is_laundering"]
    
    trx_features = data_in.select(*columns).toPandas()
    trx_features.loc[:, "inter_currency"] = trx_features["source_currency"] != trx_features["target_currency"]

    trx_features.to_parquet(location)
    del trx_features

In [28]:
%%time

save_trx_features(train, location_train_trx_features)
save_trx_features(validation, location_valid_trx_features)
save_trx_features(test, location_test_trx_features)

25/07/06 02:51:36 WARN TaskSetManager: Stage 144 contains a task of very large size (17125 KiB). The maximum recommended task size is 1000 KiB.
25/07/06 02:51:51 WARN TaskSetManager: Stage 149 contains a task of very large size (17125 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

CPU times: user 16.5 s, sys: 2.77 s, total: 19.3 s
Wall time: 2min 43s


In [29]:
def combine_features(location_features_trx, location_features_edges):
    label_columns = ["source", "target", "is_laundering"]
    columns_category = ["source_currency", "target_currency", "format"]
    new_types = {column: "category" for column in columns_category}
    new_types.update({"is_laundering": bool})
    features_input = spark.read.parquet(location_features_edges)
    trx_features_input = spark.read.parquet(location_features_trx).withColumn(
        "source_left", sf.col("source")
    ).withColumn(
        "target_left", sf.col("target")
    ).drop("source", "target")
    features_input = trx_features_input.join(
        features_input,
        (trx_features_input["source_left"] == features_input["source"]) &
        (trx_features_input["target_left"] == features_input["target"]),
        how="left"
    ).drop("source_left", "target_left")
    features_input = features_input.toPandas()
    return features_input.astype(new_types)

In [30]:
%%time

train_features = combine_features(location_train_trx_features, location_features_edges_train)
validation_features = combine_features(location_valid_trx_features, location_features_edges_valid)
test_features = combine_features(location_test_trx_features, location_features_edges_test)

25/07/06 02:52:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

CPU times: user 14.8 s, sys: 31.1 s, total: 45.8 s
Wall time: 2min 2s


In [31]:
%%time

label_columns = ["source", "target", "is_laundering"]

missing_columns = (
    (set(train_features.columns).symmetric_difference(validation_features.columns)) |
    (set(train_features.columns).symmetric_difference(test_features.columns)) |
    (set(test_features.columns).symmetric_difference(validation_features.columns))
)
for column in missing_columns:
    if missing in train_features.columns:
        print(f"Deleting missing column from train: {column}")
        del train_features[column]
    if missing in validation_features.columns:
        print(f"Deleting missing column from validation: {column}")
        del validation_features[column]
    if missing in test_features.columns:
        print(f"Deleting missing column from test: {column}")
        del test_features[column]

train_features_labels = train_features.loc[:, label_columns].copy(deep=True)
del train_features["is_laundering"]
del train_features["source"]
del train_features["target"]

validation_features_labels = validation_features.loc[:, label_columns].copy(deep=True)
validation_features = validation_features.loc[:, train_features.columns]

test_features_labels = test_features.loc[:, label_columns].copy(deep=True)
test_features = test_features.loc[:, train_features.columns]

CPU times: user 3.05 s, sys: 1.44 s, total: 4.49 s
Wall time: 4.58 s


In [32]:
print((time.time() - start) // 60)
start = time.time()

204.0


In [33]:
def f1_eval(y, y_):
    return 1 - f1_score(y, np.round(y_))


def train_model(x, y, x_, y_, cv=False):
    if cv:
        model = xgb.XGBClassifier(
            early_stopping_rounds=50, scale_pos_weight=5,
            eval_metric=f1_eval, disable_default_eval_metric=True, num_parallel_tree=1, max_depth=6,
            colsample_bytree=1, subsample=1, n_estimators=200,
            enable_categorical=True,
        )
    else:
        model = xgb.XGBClassifier(
            early_stopping_rounds=20, scale_pos_weight=7,
            eval_metric=f1_eval, disable_default_eval_metric=True, 
            num_parallel_tree=20, max_depth=6,
            colsample_bytree=0.5, subsample=0.5, 
            n_estimators=100, enable_categorical=True,
        )
    model.fit(x, y, verbose=not cv, eval_set=[(x_, y_)])
    print(f"Best iteration: {model.best_iteration}\n")
    return model

In [34]:
%%time

model = train_model(
    train_features, train_features_labels["is_laundering"].values, 
    validation_features, validation_features_labels["is_laundering"].values,
)
y_test_predicted = model.predict(test_features)
f1_final = f1_score(test_features_labels["is_laundering"], y_test_predicted) * 100
print(
    round(f1_final, 2),
    round(recall_score(test_features_labels["is_laundering"], y_test_predicted) * 100, 2)
)
print()

[0]	validation_0-f1_eval:0.95066
[1]	validation_0-f1_eval:0.85972
[2]	validation_0-f1_eval:0.66857
[3]	validation_0-f1_eval:0.52555
[4]	validation_0-f1_eval:0.47483
[5]	validation_0-f1_eval:0.45606
[6]	validation_0-f1_eval:0.44831
[7]	validation_0-f1_eval:0.44413
[8]	validation_0-f1_eval:0.44161
[9]	validation_0-f1_eval:0.43570
[10]	validation_0-f1_eval:0.43302
[11]	validation_0-f1_eval:0.43050
[12]	validation_0-f1_eval:0.42732
[13]	validation_0-f1_eval:0.42645
[14]	validation_0-f1_eval:0.42673
[15]	validation_0-f1_eval:0.42507
[16]	validation_0-f1_eval:0.42408
[17]	validation_0-f1_eval:0.42267
[18]	validation_0-f1_eval:0.42147
[19]	validation_0-f1_eval:0.41928
[20]	validation_0-f1_eval:0.41869
[21]	validation_0-f1_eval:0.41729
[22]	validation_0-f1_eval:0.41752
[23]	validation_0-f1_eval:0.41602
[24]	validation_0-f1_eval:0.41633
[25]	validation_0-f1_eval:0.41595
[26]	validation_0-f1_eval:0.41418
[27]	validation_0-f1_eval:0.41473
[28]	validation_0-f1_eval:0.41366
[29]	validation_0-f1_eva

In [35]:
%%time

CV_FOLD_PERC = 0.8
N_FOLDS = 5

f1_scores = []
for fold in range(N_FOLDS):
    print("Fold", fold + 1)
    x_train = train_features.sample(frac=CV_FOLD_PERC)
    x_train_labels = x_train.loc[:, []].join(train_features_labels, how="left")
    x_validation = validation_features.sample(frac=CV_FOLD_PERC)
    x_validation_labels = x_validation.loc[:, []].join(validation_features_labels, how="left")
    model = train_model(
        x_train, x_train_labels["is_laundering"].values, 
        x_validation, x_validation_labels["is_laundering"].values,
        cv=True
    )
    y_test_predicted = model.predict(test_features)
    f1_cv = f1_score(test_features_labels["is_laundering"], y_test_predicted) * 100
    print(
        round(f1_cv, 2),
        round(recall_score(test_features_labels["is_laundering"], y_test_predicted) * 100, 2)
    )
    f1_scores.append(f1_cv)

Fold 1
Best iteration: 134

68.81 60.5
Fold 2
Best iteration: 162

68.98 60.42
Fold 3
Best iteration: 170

68.49 60.57
Fold 4
Best iteration: 166

69.04 60.43
Fold 5
Best iteration: 128

67.67 59.97
CPU times: user 2h 8min 39s, sys: 2min 17s, total: 2h 10min 56s
Wall time: 21min 12s


In [36]:
print("GFP best: 65.69 ± 0.26")

GFP best: 65.69 ± 0.26


In [37]:
print(f"{round(f1_final, 2)} ±{round(np.std(f1_scores), 2)}")

70.18 ±0.5


In [38]:
print((time.time() - start) // 60)

38.0
